# visualizing termination time versus domain size
Tim Tyree<br>
6.11.2025

In [1]:
import os,sys,pandas as pd,numpy as np

In [25]:
from lib import *

# define module

In [26]:
# parse_iolines
def parse_iolines_single(input_fn, printing=False):
    n_input=-9999
    n_output=-9999
    with open(input_fn) as f:
#         trgt1='Printing Inputs...\n'
#         trgt2='Printing Outputs...\n'
        trgt1='Printing Inputs:\n'
        trgt2='Printing Outputs:\n'
        for n,line in enumerate(f):
            if trgt1 == line:
                if printing:
                    print(f'found inputs starting after line {n}')
                n_input=n
            if trgt2 == line:
                if printing:
                    print(f'found outputs starting after line {n}')
                n_output=n
    return n_input,n_output

In [27]:
def parse_outputs_single_trial(input_fn, n_output):
    """
    Example Usage:
TermTime = parse_outputs_single_trial(input_fn, n_output)
    """
    with open(input_fn) as f:
        for n,line in enumerate(f):
            if n == n_output+1:
                lin0 =line
                TermTime = eval(lin0[:-2].split('=')[-1])
                return TermTime
#         if n == n_output+1:
#             line_N = line
#         if n == n_output+2:
#             line_CollTime =line
# N_values=np.array([eval(s) for s in line_N[:-2].split(',')])
# CollTime_values=np.array([eval(s) for s in line_CollTime[:-2].split(',')])
# # CollTime_values=np.array([eval(s) for s in line_CollTime[:-1].split(',')])
# df=pd.DataFrame({
#     'N':N_values,
#     'CollTime':CollTime_values
# })
# df=df.loc[df.CollTime!=-9999]
# df.reset_index(inplace=True,drop=True)
## return df

In [28]:
# parse_outputs(input_fn, n_output=)
def parse_log_single(input_fn,printing=False):
    """
    Example Usage:
dict_data = parse_log_single(input_fn,printing=False)
    """
    n_input,n_output=parse_iolines_single(input_fn, printing=False)
    if n_input==-9999:
        if printing:
            print(f"Warning: n_input not found for input_fn={input_fn}.\nreturning None...")
        return None
    col_name_lst, col_value_lst=parse_inputs(input_fn,n_input,n_output,printing=printing)
    dict_inputs=dict(zip(col_name_lst,col_value_lst))
    TermTime = parse_outputs_single_trial(input_fn, n_output)
    dict_data = dict(dict_inputs)
    dict_data['TermTime'] = TermTime
    return dict_data

# dev

In [29]:
# fldr = f"{nb_dir}/data/osg_output/run_44"
# fldr = f"{nb_dir}/data/osg_output/run_45"
# fldr = f"{nb_dir}/data/osg_output/run_46"
fldr = f"{nb_dir}/data/osg_output/run_47"

In [30]:
assert os.path.exists(fldr)

In [31]:
dict_data_lst = []
for fn in os.listdir(fldr):
    if fn.find('.out.')!=-1:
        input_fn = os.path.join(fldr,fn)
        dict_data = parse_log_single(input_fn,printing=False)
        if dict_data is not None:
            #record
            dict_data_lst.append(dict_data)
df=pd.DataFrame(dict_data_lst); del dict_data_lst

In [32]:
# np.array([x for x in dict_data_lst if x is None])

In [33]:
# pd.DataFrame(dict_data_lst)

In [34]:
# len(dict_data_lst)

In [35]:
# input_fn

In [36]:
# !atom '/Users/timothytyree/Documents/GitHub/bgmc/python/data/osg_output/run_45/job.out.17477944.1912'

In [37]:
# dict_data = parse_log_single(input_fn,printing=True)
# dict_data

In [38]:
np.random.seed(42)
printing=True
if printing:
    print(f"LPM with Same Creation (Run #46):")
dict_lst = []
for (D,L),g in df.groupby(by=['D','L']):
    tau_values = g['TermTime'].values
    meanx,Delta_meanx,num_obs,p_normal=comp_mean_bootstrap_uncertainty(tau_values)
    printing=True
    a=g['varkappa'].max()
    chi=g['chi'].max()
    if printing:
        print(f"- {D=}, {L=}: mean tau: {meanx:.4f} +/- {Delta_meanx:.4f} seconds (N={num_obs}, {p_normal=:.4f})")
    dict_lst.append(dict(D=D,L=L,a=a,chi=chi,
                         tau=meanx,Delta_tau=Delta_meanx,
                         num_obs=num_obs,p_normal=p_normal,
                        ))

LPM with Same Creation (Run #46):
- D=0.048, L=3.75: mean tau: 2.5524 +/- 0.1227 seconds (N=2000, p_normal=0.7090)
- D=0.048, L=5.0: mean tau: 24.9064 +/- 0.8454 seconds (N=2000, p_normal=0.9100)
- D=0.048, L=6.25: mean tau: 57.2602 +/- 0.4532 seconds (N=2000, p_normal=0.0645)
- D=0.048, L=7.075: mean tau: 59.6768 +/- 0.1742 seconds (N=1997, p_normal=0.0032)
- D=0.048, L=8.75: mean tau: 60.0000 +/- 0.0000 seconds (N=1993, p_normal=0.0000)
- D=0.42, L=5.0: mean tau: 0.5670 +/- 0.0625 seconds (N=2000, p_normal=0.0000)
- D=0.42, L=6.25: mean tau: 1.8240 +/- 0.1634 seconds (N=2000, p_normal=0.0086)
- D=0.42, L=7.5: mean tau: 7.0232 +/- 0.4015 seconds (N=2000, p_normal=0.1577)
- D=0.42, L=8.75: mean tau: 28.3362 +/- 0.9632 seconds (N=2000, p_normal=0.4543)
- D=0.42, L=10.0: mean tau: 51.9175 +/- 0.7744 seconds (N=2000, p_normal=0.2250)


In [39]:
df_out = pd.DataFrame(dict_lst)
df_out

,D,L,a,chi,tau,Delta_tau,num_obs,p_normal
0,0.048,3.750,1.552,2.939433,2.552351,1.226709e-01,2000,7.090170e-01
1,0.048,5.000,1.552,2.939433,24.906398,8.453619e-01,2000,9.100244e-01
2,0.048,6.250,1.552,2.939433,57.260239,4.531969e-01,2000,6.449998e-02
3,0.048,7.075,1.552,2.939433,59.676798,1.741793e-01,1997,3.228690e-03
4,0.048,8.750,1.552,2.939433,60.000000,2.785328e-14,1993,0.000000e+00
5,0.420,5.000,9.300,2.075269,0.566962,6.247339e-02,2000,1.439192e-26
6,0.420,6.250,9.300,2.075269,1.823972,1.633538e-01,2000,8.571018e-03
7,0.420,7.500,9.300,2.075269,7.023152,4.014836e-01,2000,1.577064e-01
8,0.420,8.750,9.300,2.075269,28.336182,9.631525e-01,2000,4.542982e-01
9,0.420,10.000,9.300,2.075269,51.917457,7.744242e-01,2000,2.250207e-01


In [40]:
# savedir = f"{nb_dir}/data/osg_output/run_46_tau_vs_L.csv"
savedir = f"{nb_dir}/data/osg_output/run_47_tau_vs_L.csv"

In [41]:
df_out.to_csv(savedir,index=False)

In [42]:
beep(3)

-n 
-n 
-n 


0